In [440]:
import pandas as pd
import numpy as np

from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import *

from sklearn.linear_model import *
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier, XGBRegressor
from sklearn.neighbors import *

from matplotlib import pyplot as plt
import seaborn as sns

In [524]:
data = pd.read_csv('train_dataset.csv', index_col='Unnamed: 0')
test = pd.read_csv('test_dataset_v01.csv', index_col='Unnamed: 0')

In [525]:
data = data.loc[:, data.nunique() != 1]
test = test.loc[:, test.nunique() != 1]

In [526]:
data.dropna(inplace=True)

In [527]:
data['Result_v1'].replace({'benign': 1, 'malicious': -1, 1: 1, -1: -1}, inplace=True)

In [528]:
test.isna().sum()

url_len                      0
url_num_hyphens_dom          0
url_path_len               466
url_domain_len             466
url_hostname_len           463
url_num_dots                 0
url_num_underscores          0
url_query_len                0
url_num_query_para           0
url_ip_present               0
url_entropy                  0
url_port                     0
html_num_tags('iframe')      0
html_num_tags('script')    467
html_num_tags('embed')       0
html_num_tags('object')      0
html_num_tags('div')         0
html_num_tags('head')        0
html_num_tags('body')        0
html_num_tags('form')      467
html_num_tags('a')         461
dtype: int64

url_path_len
url_domain_len
url_hostname_len
html_num_tags('script')
html_num_tags('form')
html_num_tags('a')

In [529]:
data

,url_len,url_num_hyphens_dom,url_path_len,url_domain_len,url_hostname_len,url_num_dots,url_num_underscores,url_query_len,url_num_query_para,url_ip_present,...,html_num_tags('iframe'),html_num_tags('script'),html_num_tags('embed'),html_num_tags('object'),html_num_tags('div'),html_num_tags('head'),html_num_tags('body'),html_num_tags('form'),html_num_tags('a'),Result_v1
0,23.0,0.0,8.0,15.0,15.0,2.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,-1
1,75.0,0.0,58.0,17.0,17.0,6.0,0.0,0.0,0.0,0.0,...,0.0,18.0,0.0,0.0,20.0,1.0,1.0,0.0,21.0,1
2,20.0,0.0,4.0,16.0,16.0,2.0,0.0,0.0,0.0,0.0,...,1.0,33.0,0.0,0.0,101.0,1.0,1.0,3.0,70.0,1
3,27.0,0.0,13.0,14.0,14.0,3.0,0.0,0.0,0.0,0.0,...,0.0,15.0,0.0,0.0,151.0,1.0,1.0,1.0,55.0,1
4,39.0,2.0,12.0,27.0,27.0,2.0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,0.0,332.0,1.0,1.0,0.0,321.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3659,68.0,3.0,16.0,52.0,52.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,1.0,1.0,0.0,3.0,-1
3660,66.0,0.0,48.0,18.0,18.0,2.0,0.0,0.0,0.0,0.0,...,1.0,14.0,0.0,0.0,212.0,1.0,1.0,3.0,475.0,1
3661,90.0,1.0,64.0,26.0,26.0,4.0,0.0,0.0,0.0,0.0,...,0.0,13.0,0.0,0.0,75.0,1.0,1.0,2.0,103.0,-1
3662,46.0,0.0,33.0,13.0,13.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,1.0,0.0,3.0,1


In [530]:
# domain_len, hostname_len는 거의 정비례하므로 각각 대입해줌
test.loc[test.url_hostname_len.isna(),'url_hostname_len'] = test.loc[test.url_hostname_len.isna(), 'url_domain_len']
test.loc[test.url_domain_len.isna(),'url_domain_len'] = test.loc[test.url_domain_len.isna(), 'url_hostname_len']

In [531]:
# url_path_len 은 먼저 선형 회귀를 통해 집어 넣는다.

tx = data.drop(['Result_v1', 'url_path_len'], axis=1).loc[:, ['url_len', 'url_num_dots', 'url_num_underscores', 'url_domain_len', 'url_hostname_len']]
ty = data.loc[: , 'url_path_len']

model = LinearRegression().fit(tx, ty)

pred = model.predict(test.loc[(~test.url_domain_len.isna() & ~test.url_hostname_len.isna() & test.url_path_len.isna()), ['url_len', 'url_num_dots', 'url_num_underscores', 'url_domain_len', 'url_hostname_len']])
test.loc[(test.url_path_len.isna() & ~test.url_domain_len.isna() & ~test.url_hostname_len.isna()), 'url_path_len'] = pred

In [532]:
# 나머지 url_path_len은 url_len을 기준으로 소팅 후 ffill 한다
test.url_path_len = test.sort_values(by='url_len')['url_path_len'].fillna(method='ffill')
test.url_path_len.isna().sum()

0

In [533]:


test.loc[(~test.url_domain_len.isna() & ~test.url_hostname_len.isna()), "url_path_len"] = test.sort_values(by=["url_len"]).loc[: , "url_path_len"].fillna(method='ffill')

In [534]:
# html_num_tags('a') 은 html_num_tags('div') 기준으로 소팅시켜 ffill을 한다.# html_num_tags('a') 은 html_num_tags('div') 기준으로 소팅시켜 ffill을 한다.
test["html_num_tags('a')"] = test.sort_values(by=["html_num_tags('div')"]).loc[: , "html_num_tags('a')"].fillna(method='ffill')

In [535]:
# 남은 url_domian_len과 url_hostname_len은 결측치가 존재하는 컬럼을 제외하고 모두 로지스틱 회귀에 넣는다.

tx = data.drop(['Result_v1', 'url_domain_len'], axis=1).loc[:, test.isna().sum() == 0]
ty = data.loc[:, 'url_domain_len']

model = LinearRegression().fit(tx, ty)



In [536]:
pred =  model.predict(test.loc[test['url_domain_len'].isna(), test.isna().sum() == 0])
test.loc[test.url_domain_len.isna(), 'url_domain_len'] = pred
test.loc[test.url_hostname_len.isna(), 'url_hostname_len'] = pred

In [537]:
test.loc[test["html_num_tags('script')"].isna(), "html_num_tags(\'script\')"] = data["html_num_tags('script')"].mode()[0]

In [538]:
test.loc[test["html_num_tags('form')"].isna(), "html_num_tags(\'form\')"] = data["html_num_tags('form')"].mode()[0]

In [539]:
data["html_num_tags('form')"].mode()

0    1.0
Name: html_num_tags('form'), dtype: float64

In [540]:
test["html_num_tags('form')"].mode()

0    1.0
Name: html_num_tags('form'), dtype: float64

In [541]:
test.loc[test.url_domain_len.isna() | test.url_hostname_len.isna() & test["html_num_tags('script')"].isna()]

,url_len,url_num_hyphens_dom,url_path_len,url_domain_len,url_hostname_len,url_num_dots,url_num_underscores,url_query_len,url_num_query_para,url_ip_present,...,url_port,html_num_tags('iframe'),html_num_tags('script'),html_num_tags('embed'),html_num_tags('object'),html_num_tags('div'),html_num_tags('head'),html_num_tags('body'),html_num_tags('form'),html_num_tags('a')


In [542]:
test.isna().sum()

url_len                    0
url_num_hyphens_dom        0
url_path_len               0
url_domain_len             0
url_hostname_len           0
url_num_dots               0
url_num_underscores        0
url_query_len              0
url_num_query_para         0
url_ip_present             0
url_entropy                0
url_port                   0
html_num_tags('iframe')    0
html_num_tags('script')    0
html_num_tags('embed')     0
html_num_tags('object')    0
html_num_tags('div')       0
html_num_tags('head')      0
html_num_tags('body')      0
html_num_tags('form')      0
html_num_tags('a')         0
dtype: int64

In [543]:
tx = data.drop(['Result_v1', 'url_domain_len'], axis=1).loc[:, test.isna().sum() == 0]
ty = data.loc[:, 'url_domain_len']

In [544]:
tx_train, tx_val, ty_train, ty_val = train_test_split(tx, ty, test_size=.3)

In [545]:
model = LinearRegression().fit(tx_train, ty_train)

In [546]:
pred = model.predict(tx_val)

In [547]:
r2_score(ty_val, pred)

0.9987140131643593

In [548]:
print(mean_absolute_percentage_error(ty_val, pred))

0.005972570654480278


In [549]:
# params = {'max_depth': range(2, 9), 'n_estimators': [100], 'learning_rate': list(np.linspace(0.0001, 0.01, 100))} #, 'tree_method':'gpu_hist', 'gpu_id':0}
xgb_model = GridSearchCV(estimator=XGBRegressor(), param_grid=params, n_jobs=-1, cv=5, verbose=3).fit(tx_train, ty_train)


Fitting 5 folds for each of 5600 candidates, totalling 28000 fits


C:\Users\allsa\anaconda3\envs\kt-aivle\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [550]:
xgb_model.score(tx_val, ty_val)

0.9989425434452474

In [551]:
pred = xgb_model.predict(tx_val)
r2_score(ty_val, pred)

0.9989425434452474

In [552]:
print(ty_val, pred)

2014    11.0
3331    11.0
3185    13.0
2091    28.0
1368    33.0
        ... 
1394    20.0
2374    34.0
657     17.0
2697    15.0
2131    18.0
Name: url_domain_len, Length: 1099, dtype: float64 [10.94652  11.019709 13.071234 ... 16.439556 15.017884 17.993496]


In [553]:
mean_absolute_percentage_error(ty_val, pred)

0.002693819695870061

### XGB 모델

In [554]:
params = {'max_depth': range(2, 9), 'n_estimators': [20, 30, 50, 100], 'learning_rate': list(np.linspace(0.001, 0.1, 200))}

In [555]:
X = data.drop(['Result_v1'], axis=1)
y = data.loc[:, 'Result_v1']

In [556]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3)

In [557]:
params = {'max_depth': range(2, 9), 'n_estimators': [20, 30, 50, 100], 'learning_rate': list(np.linspace(0.001, 0.1, 200))}

In [558]:
xgb_model = GridSearchCV(estimator=XGBClassifier(), param_grid=params, n_jobs=-1, cv=5, verbose=2).fit(X_train, y_train)

Fitting 5 folds for each of 5600 candidates, totalling 28000 fits
[15:53:50] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\allsa\anaconda3\envs\kt-aivle\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\allsa\anaconda3\envs\kt-aivle\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [559]:
confusion_matrix(y_val, xgb_model.predict(X_val))

array([[523,  23],
       [ 19, 534]], dtype=int64)

In [560]:
print(classification_report(y_val, xgb_model.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.96      0.96      0.96       546
           1       0.96      0.97      0.96       553

    accuracy                           0.96      1099
   macro avg       0.96      0.96      0.96      1099
weighted avg       0.96      0.96      0.96      1099



In [561]:
pred = xgb_model.predict(test)

In [562]:
pred

array([ 1,  1,  1, ...,  1, -1, -1], dtype=int64)

In [563]:
submit = pd.DataFrame({'id':range(len(pred)), 'expected': pred})
submit.to_csv('submit2.csv', index=False)

In [564]:
X = data.drop(['Result_v1', "html_num_tags('form')", "html_num_tags('script')"], axis=1)
y = data.loc[:, 'Result_v1']

In [565]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3)

In [566]:
xgb_model = GridSearchCV(estimator=XGBClassifier(), param_grid=params, n_jobs=-1, cv=5, verbose=3).fit(X_train, y_train)

Fitting 5 folds for each of 5600 candidates, totalling 28000 fits
[16:01:01] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\allsa\anaconda3\envs\kt-aivle\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\allsa\anaconda3\envs\kt-aivle\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [567]:
confusion_matrix(y_val, xgb_model.predict(X_val))

array([[521,  29],
       [ 28, 521]], dtype=int64)

In [568]:
print(classification_report(y_val, xgb_model.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.95      0.95      0.95       550
           1       0.95      0.95      0.95       549

    accuracy                           0.95      1099
   macro avg       0.95      0.95      0.95      1099
weighted avg       0.95      0.95      0.95      1099



In [569]:
pred = xgb_model.predict(test.drop(["html_num_tags('form')", "html_num_tags('script')"], axis=1))

In [570]:
pred

array([ 1,  1,  1, ...,  1, -1, -1], dtype=int64)

In [571]:
submit = pd.DataFrame({'id':range(len(pred)), 'expected': pred})
submit.to_csv('submit3.csv', index=False)

In [572]:
xgb_model.best_params_

{'learning_rate': 0.09801005025125628, 'max_depth': 7, 'n_estimators': 100}

### Random Forest

In [573]:
X = data.drop(['Result_v1'], axis=1)
y = data.loc[:, 'Result_v1']

In [574]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3)

In [575]:
model = RandomForestClassifier(n_jobs=-1).fit(X_train, y_train)

In [576]:
confusion_matrix(y_val, model.predict(X_val))

array([[521,  19],
       [ 14, 545]], dtype=int64)

In [577]:
print(classification_report(y_val, model.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.97      0.96      0.97       540
           1       0.97      0.97      0.97       559

    accuracy                           0.97      1099
   macro avg       0.97      0.97      0.97      1099
weighted avg       0.97      0.97      0.97      1099



In [578]:
pred = model.predict(test)

In [579]:
submit = pd.DataFrame({'id':range(len(pred)), 'expected': pred})
submit.to_csv('submit4.csv', index=False)

#### drop columns

In [580]:
X = data.drop(['Result_v1', "html_num_tags('form')", "html_num_tags('script')"], axis=1)
y = data.loc[:, 'Result_v1']

In [581]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3)

In [582]:
model = RandomForestClassifier(n_jobs=-1).fit(X_train, y_train)

In [583]:
confusion_matrix(y_val, model.predict(X_val))

array([[485,  34],
       [ 31, 549]], dtype=int64)

In [584]:
print(classification_report(y_val, model.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.94      0.93      0.94       519
           1       0.94      0.95      0.94       580

    accuracy                           0.94      1099
   macro avg       0.94      0.94      0.94      1099
weighted avg       0.94      0.94      0.94      1099



In [585]:
pred = model.predict(test.drop(["html_num_tags('form')", "html_num_tags('script')"], axis=1))

In [586]:
submit = pd.DataFrame({'id':range(len(pred)), 'expected': pred})
submit.to_csv('submit5.csv', index=False)

#### 추가 제거 테스트

In [587]:
X = data.drop(['Result_v1', "html_num_tags('form')", "html_num_tags('script')", "html_num_tags('a')"], axis=1)
y = data.loc[:, 'Result_v1']

In [588]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3)

In [589]:
model = RandomForestClassifier(n_jobs=-1).fit(X_train, y_train)

In [590]:
confusion_matrix(y_val, model.predict(X_val))

array([[508,  47],
       [ 34, 510]], dtype=int64)

In [591]:
print(classification_report(y_val, model.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.94      0.92      0.93       555
           1       0.92      0.94      0.93       544

    accuracy                           0.93      1099
   macro avg       0.93      0.93      0.93      1099
weighted avg       0.93      0.93      0.93      1099



In [592]:
pred = model.predict(test.drop(["html_num_tags('form')", "html_num_tags('script')", "html_num_tags('a')"], axis=1))

In [593]:
submit = pd.DataFrame({'id':range(len(pred)), 'expected': pred})
submit.to_csv('submit8.csv', index=False)

### KNN

In [594]:
# pipeline 정의
pipe = make_pipeline(StandardScaler(), KNeighborsClassifier())
param = {'kneighborsclassifier__n_neighbors': [3, 4, 5, 7, 10], 'kneighborsclassifier__metric': ['minkowski', 'cityblock', 'chevyshev']}

In [595]:
X = data.drop(['Result_v1'], axis=1)
y = data.loc[:, 'Result_v1']

In [596]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3)

In [597]:
model = GridSearchCV(pipe, param_grid=param, cv=5, n_jobs=-1, ).fit(X_train, y_train)

C:\Users\allsa\anaconda3\envs\kt-aivle\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
25 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\allsa\anaconda3\envs\kt-aivle\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\allsa\anaconda3\envs\kt-aivle\lib\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\allsa\anaconda3\envs\kt-aivle\lib\site-packages\sklearn\neighbors\_classification.py", line 200, in 

In [598]:
confusion_matrix(y_val, model.predict(X_val))

array([[500,  54],
       [ 62, 483]], dtype=int64)

In [599]:
print(classification_report(y_val, model.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.89      0.90      0.90       554
           1       0.90      0.89      0.89       545

    accuracy                           0.89      1099
   macro avg       0.89      0.89      0.89      1099
weighted avg       0.89      0.89      0.89      1099



In [600]:
pred = model.predict(test)

In [601]:
submit = pd.DataFrame({'id':range(len(pred)), 'expected': pred})
submit.to_csv('submit6.csv', index=False)

In [602]:
#### drop columns

In [603]:
X = data.drop(['Result_v1', "html_num_tags('form')", "html_num_tags('script')"], axis=1)
y = data.loc[:, 'Result_v1']

In [604]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3)

In [605]:
model = GridSearchCV(pipe, param_grid=param, cv=5, n_jobs=-1, ).fit(X_train, y_train)

C:\Users\allsa\anaconda3\envs\kt-aivle\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
25 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\allsa\anaconda3\envs\kt-aivle\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\allsa\anaconda3\envs\kt-aivle\lib\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\allsa\anaconda3\envs\kt-aivle\lib\site-packages\sklearn\neighbors\_classification.py", line 200, in 

In [606]:
confusion_matrix(y_val, model.predict(X_val))

array([[479,  65],
       [ 78, 477]], dtype=int64)

In [607]:
print(classification_report(y_val, model.predict(X_val)))

              precision    recall  f1-score   support

          -1       0.86      0.88      0.87       544
           1       0.88      0.86      0.87       555

    accuracy                           0.87      1099
   macro avg       0.87      0.87      0.87      1099
weighted avg       0.87      0.87      0.87      1099



In [608]:
pred = model.predict(test.drop(["html_num_tags('form')", "html_num_tags('script')"], axis=1))

In [609]:
submit = pd.DataFrame({'id':range(len(pred)), 'expected': pred})
submit.to_csv('submit7.csv', index=False)

In [610]:
test.isna().sum()

url_len                    0
url_num_hyphens_dom        0
url_path_len               0
url_domain_len             0
url_hostname_len           0
url_num_dots               0
url_num_underscores        0
url_query_len              0
url_num_query_para         0
url_ip_present             0
url_entropy                0
url_port                   0
html_num_tags('iframe')    0
html_num_tags('script')    0
html_num_tags('embed')     0
html_num_tags('object')    0
html_num_tags('div')       0
html_num_tags('head')      0
html_num_tags('body')      0
html_num_tags('form')      0
html_num_tags('a')         0
dtype: int64